# Data overview

In [25]:
# Import dependencies for the complete notebook

import os
import config
from config import INTERACTOMES_PATH, DATA_REACTOME_PATH
from queries import QUERY_GET_ALL_GENES, QUERY_GET_ALL_PROTEINS, QUERY_GET_ALL_PROTEOFORMS
from lib.graph_database_access import get_pathways, get_query_result, make_proteoform_string, get_reactions
from lib.dictionaries import read_dictionary_one_to_set
from lib.networks import get_json_filename, create_pathway_interaction_network, read_graph, get_interactomes
import pandas as pd

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

config.set_root_wd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initial working directory: C:\git\ProteoformNetworks\src\Python
New working directory: c:\git\ProteoformNetworks


In [2]:
# Calculate number of human GENES we can get from Reactome. Genes that participate in a Reaction and a Pathway.
genes = get_query_result(QUERY_GET_ALL_GENES)
print(f"There are {len(genes)} genes.")
if not os.path.exists(DATA_REACTOME_PATH):
    os.makedirs(DATA_REACTOME_PATH)
genes["Id"].to_csv(DATA_REACTOME_PATH + "genes.csv", index=False, header=False)
genes

There are 10976 genes.


,Id
0,HSPA8
1,HDAC6
2,PARK7
3,PCNT
4,ARL13B
...,...
10971,DENND4B
10972,ALS2CL
10973,RIN2
10974,RIN1


In [3]:
# Calculate number of human Proteins we can get from Reactome. Proteins that participate in a Reaction and a Pathway.
proteins = get_query_result(QUERY_GET_ALL_PROTEINS)
print(f"There are {len(proteins)} proteins.")
proteins["Id"].to_csv(DATA_REACTOME_PATH + "proteins.csv", index=False, header=False)
proteins

There are 11074 proteins.


,Id
0,P11142
1,Q9UBN7
2,Q99497
3,O95613
4,Q3SXY8
...,...
11069,O75064
11070,Q60I27
11071,Q8WYP3
11072,Q13671


In [4]:
# Calculate number of proteoforms we can get from Reactome
proteoforms = get_query_result(QUERY_GET_ALL_PROTEOFORMS)
proteoforms['Id'] = proteoforms.apply(lambda x: make_proteoform_string(x.Id), axis=1)
print(f"These {len(proteoforms)} proteoforms represent {len(proteins)} proteins coded by {len(genes)} genes, making {len(proteoforms)/len(genes)} proteoform per gene on average. ")
proteoforms["Id"].to_csv(DATA_REACTOME_PATH + "proteoforms.csv", index=False, header=False)
proteoforms

These 14246 proteoforms represent 11074 proteins coded by 10976 genes, making 1.2979227405247813 proteoform per gene on average. 


,Id
0,A0A075B6P5;
1,A0A075B6S6;
2,A0A096LP49;
3,A0A0A6YYK7;
4,A0A0C4DH25;
...,...
14241,Q9Y6X9;
14242,Q9Y6Y8;
14243,Q9Y6Y9;
14244,"Q9Y6Y9;00160:26,00160:114"


In [5]:
# Create interactomes to make sure mapping files genes-->proteins and proteins-->proteoforms exist
interactomes = get_interactomes(config.DATA_REACTOME_PATH, INTERACTOMES_PATH)

Reading participants of all reactions for level genes...
Reading participants of all reactions for level proteins...
Reading participants of all reactions for level proteoforms...
Reading participants of all reactions for level sm...
Reading components of all complexes for level genes...
Reading components of all complexes for level proteins...
Reading components of all complexes for level proteoforms...
Reading components of all complexes for level sm...
Reading interaction network for  at genes level, method no_sm...
Reading interaction network for  at proteins level, method no_sm...
Reading interaction network for  at proteoforms level, method no_sm...
Reading interaction network for  at genes level, method with_sm...
Reading interaction network for  at proteins level, method with_sm...
Reading interaction network for  at proteoforms level, method with_sm...
Reading interaction network for  at genes level, method with_unique_sm...
Reading interaction network for  at proteins level, 

In [6]:
pathways = get_pathways()["stId"]
print(f"There are {len(pathways)} pathways.")

There are 2112 pathways.


In [7]:
reactions = get_reactions()["stId"]
print(f"There are {len(reactions)} reactions.")

There are 13806 reactions.


In [24]:
# Examples of genes with multiple proteoforms

map_genes_to_proteins = read_dictionary_one_to_set(INTERACTOMES_PATH, "mapping_proteins_to_genes.tsv", col_indices=(1, 0))
map_proteins_to_proteoforms = read_dictionary_one_to_set(INTERACTOMES_PATH, "mapping_proteins_to_proteoforms.tsv", col_indices=(0, 1))

assert len(map_genes_to_proteins.keys()) == len(genes)
assert len(map_proteins_to_proteoforms.keys()) == len(proteins)

map_genes_to_proteoforms = {}
for gene, protein_set in map_genes_to_proteins.items():
    map_genes_to_proteoforms.setdefault(gene, set())
    for protein in protein_set:
        map_genes_to_proteoforms[gene].update(map_proteins_to_proteoforms[protein])
    
map_genes_to_proteoforms

df = pd.DataFrame(map_genes_to_proteoforms.items())


{'IL6R': {'P08887-2;', 'P08887;'},
 'IL6': {'P05231;', 'P05231;00696:null'},
 'JAK2': {'O60674;',
  'O60674;00048:1007',
  'O60674;00048:1007,00048:1008',
  'O60674;00048:119,00048:221,00048:372,00048:373,00048:570,00048:637,00048:813,00048:868,00048:966,00048:972,00048:1007,00048:1008',
  'O60674;00048:813',
  'O60674;00696:null'},
 'JAK1': {'P23458;',
  'P23458;00048:1022,00048:1023',
  'P23458;00048:1034',
  'P23458;00048:null',
  'P23458;00696:null'},
 'TYK2': {'P29597;',
  'P29597;00048:1054',
  'P29597;00048:1054,00048:1055',
  'P29597;00048:null',
  'P29597;00696:null'},
 'IL6ST': {'P40189-1;00048:759,00048:767,00048:814,00048:905,00048:915',
  'P40189-2;',
  'P40189;',
  'P40189;00048:null'},
 'STAT3': {'P40763;',
  'P40763;00046:727',
  'P40763;00046:727,00048:705',
  'P40763;00046:727,00048:705,00064:685',
  'P40763;00048:705',
  'P40763;00696:null'},
 'STAT1': {'P42224-1;',
  'P42224-1;00046:727,00048:701',
  'P42224-1;00048:701',
  'P42224-2;',
  'P42224-2;00048:701',
  'P4

In [ ]:

selected_proteins = []

for gene, proteoforms in map_proteins_to_proteoforms.items():
    if len(proteoforms) > 1:
        selected_proteins.append(protein)

print(f"Only {len(selected_proteins)} have multiple proteoforms.")

#TODO: Make dataframe: Gene name, num proteoforms, proteoforms

In [9]:
selected_pathways = []
for pathway in pathways:
    filename = get_json_filename(config.proteins, config.no_sm, config.PATHWAY_GRAPHS_PATH, pathway)
    if not Path(filename).exists():
        create_pathway_interaction_network(pathway, config.proteins, config.no_sm, config.PATHWAY_GRAPHS_PATH)
    G = read_graph(filename)
    if any(protein in selected_proteins for protein in list(G.nodes)):
        selected_pathways.append(pathway)

print(f"There are {len(selected_pathways)} pathways that contain proteoforms.")

    * Creating network networks\pathways\R-HSA-9613829_proteins_no_sm.json


OSError: Cannot save file into a non-existent directory: 'networks\pathways\complexes'

In [ ]:

#  Examples when gene products participate in a different set of reactions.

In [ ]:
- Calculate set of reactions where each gene product participates
- Select genes where it's protein products have a different set of reactions
- Quantify the difference overall:
    * How often the multiple protein products participate in the same reaction
    * Quantify the intersection: 